# interface 

In [1]:
import customtkinter as ctk
from tkinter import filedialog, Label, Frame
from PIL import Image, ImageTk
import cv2
import numpy as np
import tensorflow as tf
import webbrowser

# Models path
cnn_model_path = "D:\\dataset 2\\deforestation_models\\simple CNN model for pixel-wise classification.h5"
vgg16_model_path = "D:\\dataset 2\\deforestation_models\\vgg16 model for classification fine tunning with full parameter to highlight the deforested area.h5"

# Models loading
cnn_model = tf.keras.models.load_model(cnn_model_path)
vgg16_model = tf.keras.models.load_model(vgg16_model_path)

img_path = None
selected_model = cnn_model
visualization_mode = "highlight"


def upload_image():
    global img_path
    img_path = filedialog.askopenfilename()

    if img_path:
        img = Image.open(img_path)
        img = img.resize((224, 224))
        img_tk = ImageTk.PhotoImage(img)
        img_label.config(image=img_tk)
        img_label.image = img_tk


def process_prediction(image, prediction):
    global visualization_mode
    mask = (prediction > 0.5).astype(np.uint8) * 255  # Binary mask
    mask_resized = cv2.resize(mask, (image.shape[1], image.shape[0]))

    if visualization_mode == "highlight":
        # Overlay mask on the image
        result_img = cv2.addWeighted(image, 0.6, cv2.merge([mask_resized] * 3), 0.4, 0)
#     else:  # Underline visualization
#         underline = np.zeros_like(image)
#         height, width = underline.shape[:2]

#         # Create an underline in the bottom quarter of the image
#         underline_start = int(height * 0.75)
#         underline_end = height
#         underline[underline_start:underline_end, :, 1] = mask_resized[underline_start:underline_end]  # Green underline

#         result_img = cv2.addWeighted(image, 0.8, underline, 0.2, 0)

    # Convert BGR to RGB for PIL compatibility
    result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
    result_img_pil = Image.fromarray(result_img)
    return result_img_pil


def detect_deforestation():
    global img_path, selected_model

    if not img_path:
        print("Please upload an image first!")
        return

    img = cv2.imread(img_path)
    img_resized = cv2.resize(img, (224, 224))  # Assuming input size is 224x224
    img_array = img_resized / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = selected_model.predict(img_array)[0]
    result_img = process_prediction(img, prediction)
    result_img_tk = ImageTk.PhotoImage(result_img)
    img_label.config(image=result_img_tk)
    img_label.image = result_img_tk


def toggle_model():
    global selected_model
    if selected_model == cnn_model:
        selected_model = vgg16_model
        model_label.config(text="Current Model: VGG16")
    else:
        selected_model = cnn_model
        model_label.config(text="Current Model: CNN")


def toggle_visualization_mode():
    global visualization_mode
    if visualization_mode == "highlight":
        visualization_mode = "underline"
        visualization_label.config(text="Visualization: Underline")
    else:
        visualization_mode = "highlight"
        visualization_label.config(text="Visualization: Highlight")


def open_google_search():
    webbrowser.open("https://www.google.com")  # Opens Google's homepage in the default browser


# GUI setup
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

window = ctk.CTk()
window.title("Deforestation Detection System")
window.geometry("400x750")
window.configure(bg="#f0f0f0")

header = Label(window, text="Deforestation Detection", font=("Helvetica", 20, "bold"), bg="#4682B4", fg="white", pady=10)
header.pack(fill="x")

img_frame = Frame(window, bd=2, relief="groove", bg="white")
img_frame.pack(pady=20)

img_label = Label(img_frame, bg="white")
img_label.pack(padx=10, pady=10)

# Add Upload Image button
upload_button = ctk.CTkButton(window, text="Upload Image", command=upload_image, font=("Helvetica", 12), fg_color="#87CEEB", text_color="black", corner_radius=10)
upload_button.pack(pady=10)

# Add Google Search button
google_search_button = ctk.CTkButton(
    window,
    text="Open Google Search",
    command=open_google_search,
    font=("Helvetica", 12),
    fg_color="#4169E1",
    text_color="white",
    corner_radius=10
)
google_search_button.pack(pady=10)

model_label = Label(window, text="Current Model: CNN", font=("Helvetica", 16), bg="#f0f0f0", fg="black")
model_label.pack(pady=10)

model_toggle_button = ctk.CTkButton(window, text="Change Model", command=toggle_model, font=("Helvetica", 12), fg_color="#FFA500", text_color="black", corner_radius=10)
model_toggle_button.pack(pady=10)

visualization_label = Label(window, text="Visualization: Highlight", font=("Helvetica", 16), bg="#f0f0f0", fg="black")
visualization_label.pack(pady=10)

# visualization_toggle_button = ctk.CTkButton(window, text="Change Visualization", command=toggle_visualization_mode, font=("Helvetica", 12), fg_color="#32CD32", text_color="black", corner_radius=10)
# visualization_toggle_button.pack(pady=10)

detect_button = ctk.CTkButton(window, text="Detect Deforestation", command=detect_deforestation, font=("Helvetica", 12), fg_color="#32CD32", text_color="black", corner_radius=10)
detect_button.pack(pady=10)

window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
